<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW2/blob/main/BigData_HW2_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: ‌Big Data
<br>
<b>Description:</b> HomeWork 1 | Question 2
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# Libraries

In [1]:
import pandas as pd
import numpy as np

# Dataset

In [2]:
!gdown 1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
!unzip /content/Bigdata_hw2_datasets.zip

Downloading...
From: https://drive.google.com/uc?id=1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
To: /content/Bigdata_hw2_datasets.zip
100% 6.28M/6.28M [00:00<00:00, 28.4MB/s]
Archive:  /content/Bigdata_hw2_datasets.zip
   creating: Bigdata_hw2_datasets/
   creating: Bigdata_hw2_datasets/q1/
  inflating: Bigdata_hw2_datasets/q1/stream_data_dgim.txt  
   creating: Bigdata_hw2_datasets/q2/
  inflating: Bigdata_hw2_datasets/q2/games.csv  
  inflating: Bigdata_hw2_datasets/q2/ratings.csv  
   creating: Bigdata_hw2_datasets/q3/
  inflating: Bigdata_hw2_datasets/q3/c1.txt  
  inflating: Bigdata_hw2_datasets/q3/c2.txt  
  inflating: Bigdata_hw2_datasets/q3/data.txt  


In [3]:
games_dataset = pd.read_csv("/content/Bigdata_hw2_datasets/q2/games.csv")

In [4]:
games_dataset = games_dataset[games_dataset["game_id"].notna()]

In [5]:
ratings_dataset = pd.read_csv("/content/Bigdata_hw2_datasets/q2/ratings.csv")

In [6]:
ratings_dataset = ratings_dataset.drop_duplicates(["user_id", "game_id"], keep="last")

# User-User

## Auxililary Functions

In [ ]:
def rated_item_list(user_id):
  return ratings_dataset[ratings_dataset["user_id"]==user_id]["game_id"].values

In [ ]:
def cosine_similarity(user_x, user_y):
  if user_x == user_y:
    return 1

  user_x_rates = ratings_dataset[ratings_dataset["user_id"]==user_x]["rating"].values
  user_y_rates = ratings_dataset[ratings_dataset["user_id"]==user_y]["rating"].values

  user_x_norm = sum([rate**2 for rate in user_x_rates]) ** 0.5 
  user_y_norm = sum([rate**2 for rate in user_y_rates]) ** 0.5 

  common_item_list = np.intersect1d(rated_item_list(user_x), rated_item_list(user_y))

  minimal_ratings_dataset = ratings_dataset[ratings_dataset["game_id"].isin(common_item_list) & ratings_dataset["user_id"].isin([user_x, user_y])]

  user_x_y_dot = 0
  for item_id in common_item_list:
    rate_1, rate_2 = minimal_ratings_dataset[minimal_ratings_dataset["game_id"]==item_id]["rating"].values
    user_x_y_dot += rate_1 * rate_2

  return user_x_y_dot / (user_x_norm * user_y_norm)

In [ ]:
def most_similar_users(user):
  all_user_id_list = set(ratings_dataset["user_id"].values)
  
  similarity_scores = []

  for i, user_id in enumerate(all_user_id_list):
    if i%2000==0: print(int(i*100/len(all_user_id_list)), "%")
    if user_id == user: continue

    score = cosine_similarity(user, user_id)
    similarity_scores.append({"score": score, "user-id": user_id})

  similarity_scores = sorted(similarity_scores, key=lambda x: -x["score"])
  return similarity_scores

## Prediction

In [ ]:
def predict_rate(similar_users, item_id, number):
  minimal_ratings_dataset = ratings_dataset[ratings_dataset["game_id"]==item_id]

  rater_user_id_list = minimal_ratings_dataset["user_id"].values

  sum_score = 0
  sum_rate_score = 0
  number_users = 0 

  for record in similar_users:
    user_id = record["user-id"]
    score = record["score"]
    if user_id in rater_user_id_list:
      number_users += 1
      rate = minimal_ratings_dataset[minimal_ratings_dataset["user_id"]==user_id]["rating"].values[0]
      sum_rate_score += score * rate
      sum_score += score
            
      if number_users == number:
        break

  if sum_score==0:
    return None

  rate = sum_rate_score / sum_score
  
  rate = round(rate, 3)

  return rate

## Recommendation

In [ ]:
def recommend(user_id, number_recommendation):
  similar_users = most_similar_users(user_id)

  exist_rate = ratings_dataset[ratings_dataset["user_id"] == user_id]["game_id"].values
  
  rate_list = []
  for item_id in range(1, 10001):
    if item_id%1000==0: print(int(item_id/100), "%")
    if item_id in exist_rate: continue

    rate = predict_rate(similar_users, item_id, 10)

    if rate is not None:
      rate_list.append({"rate": rate, "item-id": item_id})

  rate_list = sorted(rate_list, key=lambda x: (-x["rate"], x["item-id"]))[:number_recommendation]

  output_list = []

  for record in rate_list:
    item_id = record["item-id"]
    rate = record["rate"]

    name = games_dataset[games_dataset["game_id"] == item_id]["name"].values[0]

    output_list.append({"name": name, "game_id": item_id, "rate": rate})

  return output_list

## Results

In [ ]:
recommend(user_id=5461, number_recommendation=5)

0 %
3 %
7 %
11 %
14 %
18 %
22 %
26 %
29 %
33 %
37 %
41 %
44 %
48 %
52 %
56 %
59 %
63 %
67 %
71 %
74 %
78 %
82 %
86 %
89 %
93 %
97 %
10 %
20 %
30 %
40 %
50 %
60 %
70 %
90 %
100 %


[{'game_id': 143, 'name': 'Forza Horizon 4', 'rate': 5.0},
 {'game_id': 532, 'name': 'DUSK', 'rate': 5.0},
 {'game_id': 803, 'name': 'NASCAR 2005: Chase for the Cup', 'rate': 5.0},
 {'game_id': 1527, 'name': 'FTL: Faster Than Light', 'rate': 5.0},
 {'game_id': 1967, 'name': 'The Book of Unwritten Tales', 'rate': 5.0}]

In [ ]:
recommend(user_id=10140, number_recommendation=5)

0 %
3 %
7 %
11 %
14 %
18 %
22 %
26 %
29 %
33 %
37 %
41 %
44 %
48 %
52 %
56 %
59 %
63 %
67 %
71 %
74 %
78 %
82 %
86 %
89 %
93 %
97 %
10 %
20 %
30 %
40 %
50 %
60 %
70 %
80 %
90 %
100 %


[{'game_id': 143, 'name': 'Forza Horizon 4', 'rate': 5.0},
 {'game_id': 532, 'name': 'DUSK', 'rate': 5.0},
 {'game_id': 803, 'name': 'NASCAR 2005: Chase for the Cup', 'rate': 5.0},
 {'game_id': 1010, 'name': 'Katamari Damacy', 'rate': 5.0},
 {'game_id': 1349,
  'name': 'Shantae: Half-Genie Hero - Ultimate Edition',
  'rate': 5.0}]

# Item-Item

## Create Dictionaries

In [34]:
item_rater_dictionary = {}
item_rate_dictionary = {}
item_norm_dictionary = {}

for item_id in range(1,10001):
  if item_id%1000==0: print(f"{item_id/100}%")

  tmp_dataset = ratings_dataset[ratings_dataset["game_id"]==item_id]

  ratings = tmp_dataset["rating"].values
  
  item_norm = sum([rate**2 for rate in ratings]) ** 0.5 
  item_norm_dictionary[item_id] = item_norm

  raters = tmp_dataset["user_id"].values
  item_rater_dictionary[item_id] = raters 

  item_rates = {}
  for user_id in raters:
    item_rates[user_id] = tmp_dataset[tmp_dataset["user_id"]==user_id]["rating"].values[0] 

  item_rate_dictionary[item_id] = item_rates

10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
100.0%


## Auxililary Functions

In [35]:
def cosine_similarity_item(item_x, item_y):
  if item_x == item_y:
    return 1

  norm_x = item_norm_dictionary[item_x]
  norm_y = item_norm_dictionary[item_y]

  common_user_list = np.intersect1d(
      item_rater_dictionary[item_x], item_rater_dictionary[item_y]
  )

  item_x_y_dot = 0
  for user_id in common_user_list:
    rate_1 = item_rate_dictionary[item_x][user_id]
    rate_2 = item_rate_dictionary[item_y][user_id]
    
    item_x_y_dot += rate_1 * rate_2

  return item_x_y_dot / (norm_x * norm_y)

## Recomendation

In [56]:
def recommend_item(user_id, number_recommendation):
  user_items_id = ratings_dataset[ratings_dataset["user_id"]==user_id]["game_id"].values

  predicted_rate_list = []
  for condidate_item_id in range(1, 10001):
    if condidate_item_id%1000==0: print(condidate_item_id/100)
    if condidate_item_id in user_items_id: continue

    condidate_score_rate = []  
    for user_item_id in user_items_id:
      score = cosine_similarity_item(condidate_item_id, user_item_id)
      rate = item_rate_dictionary[user_item_id][user_id]
      condidate_score_rate.append({"score": score, "rate": rate})

    condidate_score_rate = sorted(condidate_score_rate, key=lambda x: -x["score"])[:10]

    sum_score = 0
    sum_rate_score = 0
    for item in condidate_score_rate:
      score = item["score"]
      rate = item["rate"]

      sum_rate_score += score * rate
      sum_score += score
            
    predicted_rate = sum_rate_score / sum_score
    predicted_rate_list.append({"rate": predicted_rate, "item-id": condidate_item_id})


  predicted_rate_list = sorted(predicted_rate_list, key=lambda x: (-x["rate"], x["item-id"]))[:number_recommendation]

  output_list = []

  for record in predicted_rate_list:
    item_id = record["item-id"]
    rate = record["rate"]

    name = games_dataset[games_dataset["game_id"] == item_id]["name"].values[0]

    output_list.append({"name": name, "game_id": item_id, "rate": rate})

  return output_list

## Results

In [57]:
recommend_item(5461, 5)

10.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0


[{'game_id': 1352, 'name': 'F1 2018', 'rate': 5.0},
 {'game_id': 184,
  'name': "Sid Meier's Alpha Centauri",
  'rate': 4.905536177685458},
 {'game_id': 161, 'name': 'Vagrant Story', 'rate': 4.889751746479591},
 {'game_id': 158, 'name': 'Forza Motorsport', 'rate': 4.888943107893666},
 {'game_id': 964, 'name': 'NHL 12', 'rate': 4.811117077816452}]

In [58]:
recommend_item(10140, 5)

10.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0


[{'game_id': 185, 'name': 'Wave Race 64', 'rate': 3.9455071741670147},
 {'game_id': 673, 'name': 'Night in the Woods', 'rate': 3.9053010975924574},
 {'game_id': 3061,
  'name': 'Wargame: AirLand Battle',
  'rate': 3.833685818880154},
 {'game_id': 2681, 'name': 'Etrian Odyssey Nexus', 'rate': 3.8243616343253675},
 {'game_id': 1210, 'name': 'Assetto Corsa', 'rate': 3.8077678042517737}]